<a href="https://colab.research.google.com/github/pruthvireddy/abacus/blob/main/polygon_stock_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Files

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
%cd /content/drive/My Drive/Github/abacus

/content/drive/My Drive/Github/abacus


In [46]:
!git config --global user.name "Pruthvi Panati"
!git config --global user.email "pruthvi5007@gmail.com"

# Setup Project

In [22]:
# This code retrieves stock market data for a specific stock using the
# Polygon REST API and writes it to a CSV file. It uses the "polygon"
# library to communicate with the API and the "csv" library to write
# the data to a CSV file. The script retrieves data for the stock "AAPL"
# for the dates "2023-01-30" to "2023-02-03" in 1 hour intervals. The
# resulting data includes the open, high, low, close, volume, vwap,
# timestamp, transactions, and otc values for each hour. The output is
# then printed to the console.
from polygon import RESTClient
from polygon.rest.models import (
    Agg,
)
import pandas as pd
import csv
import datetime
import io

# docs
# https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to
# https://polygon-api-client.readthedocs.io/en/latest/Aggs.html#polygon.RESTClient.list_aggs

client = RESTClient("ImKB1_hq_ZqfULYOdd2ANBgQNyzKWrWH") # hardcoded api_key is used
#client = RESTClient()  # POLYGON_API_KEY environment variable is used


# Query API for Data

In [24]:
def query_aggregates_to_dataframe(
    client,
    ticker: str,
    start_date: str,
    end_date: str,
    interval: int = 1,
    timespan: str = "hour"
) -> pd.DataFrame:
    """
    Query the API for aggregates of a specific ticker symbol and return data as a pandas DataFrame
    with separate date and time columns.

    Args:
        client: The API client object.
        ticker: The ticker symbol to query (e.g., "AAPL").
        start_date: The start date for the query in "YYYY-MM-DD" format.
        end_date: The end date for the query in "YYYY-MM-DD" format.
        interval: The aggregation interval (default is 1).
        timespan: The aggregation timespan (e.g., "hour", default is "hour").

    Returns:
        A pandas DataFrame containing the aggregated data. Returns an empty DataFrame if no data is retrieved.
    """
    aggs = []
    for agg in client.list_aggs(ticker, interval, timespan, start_date, end_date, limit=50000):
        aggs.append(agg)

    # Prepare data for DataFrame
    data = []
    for agg in aggs:
        if isinstance(agg, Agg) and isinstance(agg.timestamp, int):
            data.append({
                "timestamp": datetime.datetime.fromtimestamp(agg.timestamp / 1000),
                "date": datetime.datetime.fromtimestamp(agg.timestamp / 1000).date(),
                "time": datetime.datetime.fromtimestamp(agg.timestamp / 1000).time(),
                "open": agg.open,
                "high": agg.high,
                "low": agg.low,
                "close": agg.close,
                "volume": agg.volume,
                "vwap": agg.vwap,
                "transactions": agg.transactions,
                "otc": agg.otc,
            })

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    return df

In [37]:
# Initialize an empty DataFrame to store the data
#tqqq_hourly_historical = pd.DataFrame()


# Fetch the data for the year
yearly_data = query_aggregates(
    client=client,
    ticker="TQQQ",
    start_date="2024-01-01",
    end_date="2024-12-31"
)

In [38]:

# Append the data to the main DataFrame
tqqq_hourly_historical = pd.concat([tqqq_hourly_historical, yearly_data], ignore_index=True)


In [39]:
tqqq_hourly_historical

,timestamp,open,high,low,close,volume,vwap,transactions,otc
0,2023-01-10 09:00:00,17.9500,18.0300,17.86,18.03,617594.0,17.9424,3668,None
1,2023-01-10 10:00:00,18.0300,18.1100,17.97,18.08,276479.0,18.0531,2748,None
2,2023-01-10 11:00:00,18.0800,18.0900,17.78,17.83,532070.0,17.9019,4715,None
3,2023-01-10 12:00:00,17.8300,17.8300,17.57,17.68,1704808.0,17.6766,7698,None
4,2023-01-10 13:00:00,17.8200,18.0985,17.57,17.73,4129057.0,17.6708,11865,None
...,...,...,...,...,...,...,...,...,...
7922,2024-12-31 20:00:00,79.6300,80.1700,78.75,79.12,8632997.0,79.4571,58035,None
7923,2024-12-31 21:00:00,79.1100,79.4298,78.86,79.26,551812.0,79.2615,3799,None
7924,2024-12-31 22:00:00,79.2799,79.3700,79.17,79.17,140587.0,79.2727,1398,None
7925,2024-12-31 23:00:00,79.1799,79.1799,79.05,79.10,95928.0,79.1141,1002,None


# Save the Data

In [42]:
folder_path = '/content/drive/My Drive/Github/abacus/data/'

# Specify the file path
file_path = folder_path + 'tqqq_2023_2024.csv'

# Write DataFrame to CSV
tqqq_hourly_historical.to_csv(file_path, index=False)
print(f"CSV saved at: {file_path}")

CSV saved at: /content/drive/My Drive/Github/abacus/data/tqqq_2023_2024.csv


In [44]:
!git add .
!git commit -m "Update from Colab"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [45]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address
